Aim: compute QCRB for different lossless systems with input-output transfer functions of the form

$$
T_{uy}(\omega) = \prod_{j=0}^n \frac{\omega + \Delta_j + i\gamma_j}{\omega + \Delta_j - i\gamma_j},
$$

where $\gamma_j \in \mathbb{R}$ is the bandwidth for each mode and $\Delta_j \in \mathbb{R}$ is the detuning. Then calculate QCRB for each internal mode,

$$
\sigma_{xx}^\text{QCRB}(\omega) = \frac{\hbar^2}{4 \bar{S}_{FF}(\omega)} = \frac{\hbar^2}{4 |T_{uF}(\omega)|^2},
$$

assuming measurement shot noise at the input and where $T_{uF}$ is the transfer function from the input $\hat{u}$ to an internal degree of freedom $\hat{F}$ which is coupled to a classical signal $x(t)$ via $\hat{H}_\text{int} = - \hat{F} x$.

So for each $\hat{F}$ want to optimise with respect to $\gamma_j, \Delta_j, \forall j$, then choose the $\hat{F}$ that maximises the QCRB.

In [364]:
%load_ext autoreload
%autoreload
from simba import transfer_function_to_graph, tf2rss, adiabatically_eliminate
from sympy import symbols, simplify, Matrix, sqrt,\
    conjugate, lambdify, I, pi, fraction, solve, Eq, limit
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## $n = 1$ Detuned Resonator

Note: due to how the transformation to a realisable state space is performed, we have to treat the negative bandwidth and positive bandwidth separately. Need to fix at some point (else will need to perform the transformation $2^n$ times for an $n$ degree-of-freedom system for all possible combinations)

In [251]:
s = symbols('s')
gamma_f = symbols('gamma_f', real=True, positive=True)
Delta = symbols('Delta', real=True)
tf = (s + I * Delta + gamma_f) / (s + I * Delta - gamma_f)

split_network = tf2rss(tf).to_slh().split()
gamma, = split_network.aux_coupling_constants

In [252]:
split_network.state_vector

Matrix([
[              a],
[   conjugate(a)],
[             a'],
[  conjugate(a')],
[            ain],
[ conjugate(ain)],
[           aout],
[conjugate(aout)]])

In [253]:
tfm = split_network.tfm
input_output = tfm.open_loop('l', 'aout').simplify()
adiabatically_eliminate(input_output, gamma)

(-I*Delta + gamma_f + s)/(I*Delta + gamma_f - s)

In [290]:
Omega = symbols('Omega')
tf_a = tfm.open_loop('ain', 'a').simplify().subs(s, I * Omega)
tf_a

sqrt(2)*gamma*sqrt(gamma_f)/(gamma*gamma_f + (I*Delta - I*Omega)*(-I*Omega + gamma))

In [294]:
tf = adiabatically_eliminate(tf_a, gamma)
tf

sqrt(2)*sqrt(gamma_f)/(I*Delta - I*Omega + gamma_f)

In [295]:
numer, denom = fraction(tf)
solns = solve(denom, Omega)[0]
solns

Delta - I*gamma_f

We want to evaluate

$$
\int_{-\infty}^{\infty} d\Omega \frac{2 \gamma_f}{(\Omega - \Delta - i \gamma_f)(\Omega - \Delta + i \gamma_f)}
$$

Since the modulus of the integrand $\to 0$ as $\Omega \to \infty$ then we can use Jordan's lemma and this integral is equivalent to,

$$
\oint_C dz \frac{2 \gamma_f}{(z - \Delta - i \gamma_f)(z - \Delta + i \gamma_f)}
$$

where $C$ is an anti-clockwise contour containing the entire upper-half plane.

The function has one simple pole in the upper-half plane: $z = \Delta + i \gamma_f$. The residue of the integrand here is $-i$.

So the integral is

$$
\int_{-\infty}^{\infty} d\Omega \frac{2 \gamma_f}{(\Omega - \Delta - i \gamma_f)(\Omega - \Delta + i \gamma_f)} = 2 \pi
$$

and so it is independent of detuning and bandwidth (Mizuno limit)

## $n = 2$ Coupled Cavity

In [302]:
gamma_f, omega_s = symbols('gamma_f omega_s', real=True, positive=True)
tf = (s**2 + s * gamma_f + omega_s**2) / (s**2 - s * gamma_f + omega_s**2)
split_network = tf2rss(tf).to_slh().split()
tfm = split_network.tfm
gamma_1, _ = split_network.aux_coupling_constants

In [303]:
split_network.state_vector.T

Matrix([[a_1, conjugate(a_1), a_2, conjugate(a_2), a'_1, conjugate(a'_1), a'_2, conjugate(a'_2), ain_1, conjugate(ain_1), aout_1, conjugate(aout_1), ain_2, conjugate(ain_2), aout_2, conjugate(aout_2)]])

In [305]:
tf = tfm.open_loop('ain_1', 'a_1').simplify()
tf_1 = adiabatically_eliminate(tf, gamma_1).simplify().subs(s, I * Omega)
tf = tfm.open_loop('ain_1', 'a_2').simplify()
tf_2 = adiabatically_eliminate(tf, gamma_1).simplify().subs(s, I * Omega)

In [306]:
tf_1

-sqrt(2)*I*Omega*sqrt(gamma_f)/(-Omega**2 - I*Omega*gamma_f + omega_s**2)

In [307]:
tf_2

sqrt(2)*sqrt(gamma_f)*omega_s/(-Omega**2 - I*Omega*gamma_f + omega_s**2)

In [322]:
numer, denom = fraction(tf_1)
solns = solve(denom, Omega)
Matrix(solns)

Matrix([
[-I*gamma_f/2 - sqrt(-gamma_f**2 + 4*omega_s**2)/2],
[-I*gamma_f/2 + sqrt(-gamma_f**2 + 4*omega_s**2)/2]])

In [327]:
abs(tf_1)**2

2*gamma_f*Abs(Omega/(Omega**2 + I*Omega*gamma_f - omega_s**2))**2

Note that it goes to zero as $\Omega \to \infty$

For now we assume that $C^2 \equiv -\gamma_f^2 + 4 \omega_s^2 > 0$.

We want to evaluate

$$
\int_{\infty}^{\infty} 2 \gamma_f \frac{d\Omega \Omega^2}{(\Omega - i \gamma_f/2 - C/2)(\Omega - i \gamma_f/2 + C/2)(\Omega + i \gamma_f/2 - C/2)(\Omega + i \gamma_f/2 + C/2)}
$$

Follow same method as before. We have two simple poles in upper half plane: $z = i \gamma_f / 2 + C/2$, $z = i \gamma_f / 2 - C/2$.

The residues are $-i/2 \left(\frac{i \gamma_f}{C} + 1\right)$ and $i/2 \left(\frac{i \gamma_f}{C} - 1\right)$

So the integral is equal to $2 \pi$ and the Mizuno limit is not surpassed.

Now will consider $-C^2 \equiv -\gamma_f^2 + 4 \omega_s^2 < 0$ so the integral is instead,

$$
\int_{\infty}^{\infty} 2 \gamma_f \frac{d\Omega \Omega^2}{(\Omega - i \gamma_f/2 - i C /2)(\Omega - i \gamma_f/2 + i C /2)(\Omega + i \gamma_f/2 - i C /2)(\Omega + i \gamma_f/2 + i C /2)}
$$

Without loss of generality we will choose the positive root so that $C > 0$, so that $C /2 = \sqrt{\gamma_f^2 / 4 - \omega_s^2} < \gamma_f / 2$. So we have two simple poles in the upper-half plane: $z = i \gamma_f / 2 + i C / 2$ and $z = i \gamma_f / 2 - i C / 2$.

The residues are $-i/2 (\gamma_f / C + 1)$ and $i/2 (\gamma_f / C- 1)$ and so, again, the integral is equal to $2 \pi$.

In [341]:
abs(tf_2)**2

2*gamma_f*omega_s**2*Abs(1/(Omega**2 + I*Omega*gamma_f - omega_s**2))**2

Same process for other degree of freedom:

$$
\int_{\infty}^{\infty} 2 \gamma_f \omega_s^2 \frac{d\Omega}{(\Omega - i \gamma_f/2 - C/2)(\Omega - i \gamma_f/2 + C/2)(\Omega + i \gamma_f/2 - C/2)(\Omega + i \gamma_f/2 + C/2)}
$$

Same simple poles $z = i \gamma_f / 2 + C/2$, $z = i \gamma_f / 2 - C/2$. Residues are

$$
-i 2 \omega_s^2 \frac{1}{C(i \gamma_f +C)}
$$

$$i 2 \omega_s^2 \frac{1}{C(i \gamma_f - C)}$$

The sum is $$-i 2 \omega_s^2 (\frac{1}{C(i \gamma_f +C)} - \frac{1}{C(i \gamma_f - C)})$$

$$-2 i \omega_s^2 \frac{(i \gamma_f - C) - (i \gamma_f + C)}{C(i \gamma_f + C)(i \gamma_f - C)} = -2 i \omega_s^2 \frac{-2}{-\gamma_f^2 - C^2} = -2 i \omega_s^2 \frac{2}{4\omega_s^2} = -i.$$

So the integral is equal to $2 \pi$ as expected.

I will assume it is the same for the case where $\gamma_f > 4 \omega_s^2$.

## $n = 2$ Active Coupled Cavity

In [392]:
# parameterise with lambda = g**2 - omega_s**2 > 0
lmbda = symbols('lambda', real=True, positive=True)
tf = (s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)
split_network = tf2rss(tf).to_slh().split()
tfm = split_network.tfm

In [393]:
tf = tfm.open_loop('ain_1', 'a_1').simplify()
tf_1 = adiabatically_eliminate(tf, gamma_1).simplify().subs(s, I * Omega)
tf = tfm.open_loop('conjugate(ain_1)', 'a_2').simplify()
tf_2 = adiabatically_eliminate(tf, gamma_1).simplify().subs(s, I * Omega)

In [394]:
abs(tf_1)**2

2*gamma_f*Abs(Omega/(Omega**2 + I*Omega*gamma_f + lambda))**2

In [395]:
numer, denom = fraction(tf_1)
solns = solve(denom, Omega)
Matrix(solns)

Matrix([
[I*(-gamma_f + sqrt(gamma_f**2 + 4*lambda))/2],
[-I*(gamma_f + sqrt(gamma_f**2 + 4*lambda))/2]])

Note that square root is always positive here, so the root is always pure imaginary. Let us calculate the residue of the 2 simple poles that lie in the upper-half plane.

In [405]:
integrand = (2*gamma_f*abs(Omega)**2)/ \
    ((Omega + solns[0])*(Omega + solns[0].conjugate())*
     (Omega + solns[1])*(Omega + solns[1].conjugate()))

def get_residue(expr, c):
    return (expr * (Omega - c)).subs(Omega, c).simplify()

first_residue = get_residue(integrand, solns[0])
second_residue = get_residue(integrand, solns[1])

2 * pi * I * (first_residue + second_residue)

2*pi

No improvement over Mizuno limit when coupling signal into first cavity

Let's look at the transfer function from $a_\text{in}^\dagger$ to $a_2$

In [397]:
abs(tf_2)**2

2*gamma_f*lambda*Abs(1/(Omega**2 + I*Omega*gamma_f + lambda))**2

Roots are the same again.

In [404]:
integrand = (2*gamma_f*lmbda)/ \
    ((Omega + solns[0])*(Omega + solns[0].conjugate())*
     (Omega + solns[1])*(Omega + solns[1].conjugate()))

first_residue = get_residue(integrand, solns[0])
second_residue = get_residue(integrand, solns[1])

2 * pi * I * (first_residue + second_residue).simplify()

2*pi